In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers

In [125]:
dataset_dir = "Datasets/winequality-white.csv"

In [126]:
df = pd.read_csv(dataset_dir, sep = ";", index_col=None)
df = df.fillna(df.mean())
df['quality'] = (df['quality'] >= 6).astype(int)
label_column = df.pop('quality')
label_column = np.asarray(label_column)
df.insert(0, 'label', label_column)
df

,label,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8
1,1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5
2,1,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1
3,1,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
4,1,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,1,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2
4894,0,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6
4895,1,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4
4896,1,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8


In [127]:
df.isnull().values.any()

False

In [128]:
df.to_csv("./Datasets/white_wine_binary.csv")

In [99]:
df = df.sample(frac=1, random_state=41)

In [100]:
col_num = len(df.columns)
row_num = len(df.index)
print("Number of columns is: {} and number of rows is: {}".format(col_num, row_num))

Number of columns is: 12 and number of rows is: 4898


In [101]:
trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

In [102]:
scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
textx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [84]:
def get_MICS_model(inp_size, drop_out, hidden_num = 3, hidden_size=256):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    h = keras.layers.BatchNormalization()(h)
    for hidden in range(hidden_num):
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
        h = keras.layers.Dropout(drop_out)(h) 
        h = keras.layers.BatchNormalization()(h)

    outputs = keras.layers.Dense(1, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [120]:
def get_MICS_model(inp_size, drop_out, hidden_num = 3, hidden_size=256):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    h = keras.layers.BatchNormalization()(h)
    h = keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h)
    h = keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h)    
    h = keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h) 
    h = keras.layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h) 
    
    outputs = keras.layers.Dense(1, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [121]:
df["label"].value_counts()

1    3258
0    1640
Name: label, dtype: int64

In [123]:
inp_size = len(trainx.columns)
MICS_model = get_MICS_model(inp_size, drop_out = 0.25)
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25), 
        keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.1, patience=10,
                                         verbose = 2, mode = "auto", 
                                          min_lr = 1e-5)]
MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(), metrics = ["accuracy"])
history = MICS_model.fit(x = [trainx_scaled], y = trainy.values,  
                         validation_data = ([textx_scaled], testy.values),
                         epochs=300, batch_size = 32, callbacks=callback)
training_val_accuracy = history.history["val_accuracy"]
best_row_index = np.argmax(training_val_accuracy)
best_val_accuracy = training_val_accuracy[best_row_index]
best_val_accuracy
#min_losses3[c].append(best_val_loss)
#i = i + 1

Epoch 1/300
123/123 [==============================] - 1s 6ms/step - loss: 1.9397 - accuracy: 0.7070 - val_loss: 1.3242 - val_accuracy: 0.7102
Epoch 2/300
123/123 [==============================] - 1s 4ms/step - loss: 1.0498 - accuracy: 0.7356 - val_loss: 0.8814 - val_accuracy: 0.7398
Epoch 3/300
123/123 [==============================] - 0s 4ms/step - loss: 0.9284 - accuracy: 0.7399 - val_loss: 0.9427 - val_accuracy: 0.6786
Epoch 4/300
123/123 [==============================] - 1s 4ms/step - loss: 0.8877 - accuracy: 0.7210 - val_loss: 0.8989 - val_accuracy: 0.7265
Epoch 5/300
123/123 [==============================] - 1s 6ms/step - loss: 0.8762 - accuracy: 0.7340 - val_loss: 0.8130 - val_accuracy: 0.7561
Epoch 6/300
123/123 [==============================] - 1s 4ms/step - loss: 0.8547 - accuracy: 0.7305 - val_loss: 0.8808 - val_accuracy: 0.7480
Epoch 7/300
123/123 [==============================] - 1s 5ms/step - loss: 0.8480 - accuracy: 0.7437 - val_loss: 0.8303 - val_accuracy: 0.7786

KeyboardInterrupt: 

In [ ]:
df["label"].value_counts()